In [5]:
# Load the data into Pandas dataframe
import pandas as pd
import numpy as np

movies_filename = 'https://raw.githubusercontent.com/alex-coch/Recommender-systems-review/main/movies.csv'
ratings_filename = '/content/gdrive/MyDrive/rating.csv'

from google.colab import drive
drive.mount('/content/gdrive')

df_movies = pd.read_csv(
    movies_filename,
    usecols=['movieId', 'title'],
    dtype={'movieId': 'int32', 'title': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    usecols=['userId', 'movieId', 'rating'],
    dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
df_movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [7]:
df_ratings.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


In [8]:
# Pivot ratings onto movie features
# Pivot the dateframe to have userId as rows amd movieId as columns, filling the null values with 0.0
df_ratings=df_ratings[:2000000]
df_movie_features = df_ratings.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).fillna(0)

In [9]:
df_movie_features.head()



movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,128671,128686,128715,128736,128832,128842,128898,128900,128902,128968,128991,129030,129034,129036,129068,129233,129235,129303,129350,129354,129428,129530,129659,129707,129786,129788,129822,129857,130052,130069,130073,130075,130219,130462,130490,130496,130512,130642,130644,130768
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
R = df_movie_features.values
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

# Applaying singular value decomposition
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)

# Done. 
# that the Sigma$ returned is just the values instead of a diagonal matrix. 
# This is useful, but since I'm going to leverage matrix multiplication to get predictions 
# I'll convert it to the diagonal matrix form.
sigma = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = df_movie_features.columns)

In [11]:
preds_df.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,128671,128686,128715,128736,128832,128842,128898,128900,128902,128968,128991,129030,129034,129036,129068,129233,129235,129303,129350,129354,129428,129530,129659,129707,129786,129788,129822,129857,130052,130069,130073,130075,130219,130462,130490,130496,130512,130642,130644,130768
0,-1.032377,0.816787,0.065366,-0.069286,-0.089830,0.488379,-0.417959,0.045865,-0.139050,-0.341772,-0.513422,0.178584,0.069932,-0.034303,-0.045440,0.260363,0.069652,0.285616,0.471105,-0.199864,-0.034730,0.439173,-0.025721,0.213959,0.094146,0.060702,0.057206,-0.044555,1.233433,-0.004471,0.095023,2.282895,-0.011613,0.456060,0.021102,0.520967,0.004015,-0.014233,0.176869,0.043323,...,-0.007710,-0.007637,-0.001437,-0.001974,-0.001901,-0.002341,-0.007710,-0.010073,-0.009284,-0.003201,-0.010073,-0.002748,-0.001605,-0.001605,-0.009607,-0.000441,-0.004316,-0.003167,0.000509,-0.005256,-0.007290,-0.001605,-0.014006,-0.002858,-0.001877,-0.001741,-0.001728,-0.004781,-0.004179,-0.011247,0.000436,-0.000913,-0.005743,-0.006718,-0.004173,-0.002836,-0.009607,-0.006603,-0.001093,-0.002193
1,0.878418,0.018588,0.350089,0.069685,0.219142,0.379299,0.404332,0.008962,0.102663,-0.336462,0.312137,0.088314,0.029474,0.091948,0.045356,0.030740,0.429898,0.061929,-0.042158,0.062091,-0.037035,0.128742,0.110055,0.276819,0.333761,0.034210,0.057615,-0.010574,0.326948,-0.026605,0.111463,1.481748,0.002342,0.073128,-0.019877,0.147413,-0.003927,0.020103,-0.161433,-0.000244,...,-0.003447,0.000291,-0.001468,-0.000321,0.000662,-0.000419,-0.003447,0.001479,0.001308,0.001028,0.001479,0.000548,-0.000801,-0.000801,-0.004802,0.001637,0.000272,0.001330,-0.000974,-0.003719,-0.003486,-0.000801,0.001901,0.000244,-0.000653,-0.000727,0.000495,-0.001496,0.000349,0.001411,0.001779,-0.000374,-0.001408,0.002510,0.002424,-0.000837,-0.004802,0.001166,-0.000476,0.000287
2,2.004509,0.853212,-0.124892,0.033102,-0.197887,0.725844,-0.070364,-0.043636,-0.004803,0.117874,0.675620,-0.011990,0.023256,0.035115,-0.085638,1.054158,-0.394640,-0.095323,-0.392497,-0.006020,0.612250,0.300114,-0.104179,0.892096,0.300976,-0.106821,0.052140,-0.153646,0.799834,-0.006502,-0.099880,2.957936,0.010231,0.690257,-0.046066,-0.073031,0.025883,0.007296,0.748279,-0.027530,...,0.014029,0.002355,-0.005867,0.000009,-0.003435,-0.000388,0.014029,-0.001359,-0.001174,0.000396,-0.001359,0.000377,0.000973,0.000973,0.019518,0.010235,0.000436,0.002968,-0.000737,0.005804,-0.004255,0.000973,0.003951,0.000342,0.000839,0.000906,0.000638,-0.003460,-0.000256,0.003040,0.003959,-0.004786,-0.001086,0.001811,0.005158,0.001003,0.019518,-0.001382,-0.001391,0.000514
3,-0.730042,0.575875,0.316375,-0.043770,0.175544,0.804843,0.024102,0.053929,0.182834,1.027779,0.215089,0.053206,0.031952,0.014743,0.069392,0.533745,-0.540016,0.039601,0.485803,0.259154,0.650261,0.308804,0.244486,0.137633,-0.052208,-0.045443,-0.014521,-0.067064,0.017106,-0.024689,0.171829,0.444830,-0.000832,0.061156,-0.058791,-0.131904,0.003973,0.000392,0.206131,-0.037648,...,0.001746,0.001691,0.002540,0.001675,0.002820,0.001738,0.001746,0.004465,0.004178,0.002712,0.004465,0.001991,0.001990,0.001990,0.001691,0.001270,0.001783,0.002540,0.001151,0.001958,0.004704,0.001990,0.001713,0.001938,0.001969,0.001979,0.001728,0.002784,0.001792,0.001756,0.001220,0.001608,0.002619,0.001078,0.003182,0.001051,0.001691,0.001607,0.002382,0.001786
4,1.487831,1.324306,1.370973,0.112845,1.282828,0.726826,1.453526,0.217864,0.285134,1.278285,1.579988,0.208238,0.088085,0.475250,0.035512,-0.176552,1.851011,0.099257,0.402609,0.031267,0.674253,0.260641,0.019003,0.071229,0.663681,0.228992,0.160756,0.229393,0.067370,-0.015628,0.448435,0.985231,-0.006345,2.396324,0.040164,1.618643,0.005433,0.037594,0.310420,0.106339,...,-0.002487,-0.002947,0.004797,0.000543,0.002156,0.000243,-0.002487,0.003953,0.003598,0.002265,0.003953,0.001517,-0.001509,-0.001509,-0.003786,0.000288,0.002172,-0.001100,0.000004,0.001105,-0.005823

In [12]:
# Making recommendations for userID
def recommend_movies(preds_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False) # UserID starts at 1
    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.userId == (userID)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies_df[~movies_df['movieId'].isin(user_full['movieId'])]).merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left', left_on = 'movieId',
               right_on = 'movieId').rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :-1]
    return user_full, recommendations


already_rated, predictions = recommend_movies(preds_df, 330, df_movies, df_ratings, 10)

In [17]:
# The estimate of rating for the all client-items pairs 
already_rated.head(10)

,userId,movieId,rating,title
11,330,17,5.0,Sense and Sensibility (1995)
28,330,47,5.0,Seven (a.k.a. Se7en) (1995)
30,330,50,5.0,"Usual Suspects, The (1995)"
200,330,588,5.0,Aladdin (1992)
154,330,349,5.0,Clear and Present Danger (1994)
190,330,509,5.0,"Piano, The (1993)"
44,330,110,5.0,Braveheart (1995)
131,330,296,5.0,Pulp Fiction (1994)
0,330,1,4.0,Toy Story (1995)
183,330,457,4.0,"Fugitive, The (1993)"


In [18]:
print(predictions)

     movieId                                              title
159      293  Léon: The Professional (a.k.a. The Professiona...
178      329                      Star Trek: Generations (1994)
397      608                                       Fargo (1996)
187      342                            Muriel's Wedding (1994)
304      497                      Much Ado About Nothing (1993)
99       163                                   Desperado (1995)
200      368                                    Maverick (1994)
280      468  Englishman Who Went Up a Hill But Came Down a ...
130      223                                      Clerks (1994)
1          5                 Father of the Bride Part II (1995)


[Назад к обзору](https://colab.research.google.com/drive/10K5btcvrqwWm1I2rPWJJ0jpqLnVHz6Im)